# Indexing blog posts for a blog roll

This notebooks aggregates the `deathbeds` modules into a tidy `pandas.DataFrame`.  That object can be used to created a structured view of the posts.

In [25]:
    from pathlib import Path
    import deathbeds
    from pandas import DataFrame, Series, Index, to_datetime
    from IPython.display import Markdown, display
    loader = __import__('importnb').Execute(display=True);

* glob the modules in the `deathbeds` package.

In [26]:
    modules = Series(index=Index(Path(deathbeds.__file__).parent.glob('*.ipynb')));

Convert a file path to and imported module

In [27]:
    def file_to_module(path):
        with loader:
            name = path.relative_to(Path(deathbeds.__file__).parent).stem.split('.', 1)[0]
            return getattr(__import__('.'.join(('deathbeds', name))), name)

* Import all of the modules.  As the modules are imported we are testing their efficacy when `__name__ != '__main__`

In [28]:
    for path in modules.index: modules.loc[path] = file_to_module(path)

* Tidy the module information to create a blog roll

In [29]:
    df = modules.to_frame('module')
    df['docstring'] = df['module'].apply(lambda x: x.__doc__)
    df['name'] = df['module'].apply(lambda x: x.__name__)
    df['title'] = df.docstring.str.lstrip('#').str.lstrip().apply(lambda x: x.splitlines()[0])
    df['relative'] = df.index.map(lambda x: f'deathbeds/{x.relative_to(Path(deathbeds.__file__).parent)}')

    df['date'] = df['name'].str.split('.', 1).apply(lambda _: '-'.join(_[1].split('-', 3)[:3])).pipe(
        lambda s: s[s.apply(lambda x: x[0]).str.isnumeric()]
    ).pipe(to_datetime)


    blog_roll = Markdown(df.dropna(subset=['date']).sort_values('date', ascending=False).apply(
        lambda _: f"""* ## [{_.loc['title']}]({_.loc['relative']})

    > {_.loc['date'].strftime("%A  %B %d, %Y")}

    """, axis=1
    ).pipe("".join));

In [31]:
    df.sample(2).reset_index(drop=True)

,module,docstring,name,title,relative,date
0,<module 'deathbeds.2018-07-07-A-Jinja2-Templat...,# Templating input cells with [`jinja2`](http:...,deathbeds.2018-07-07-A-Jinja2-Templating-Trans...,Templating input cells with [`jinja2`](http://...,deathbeds/2018-07-07-A-Jinja2-Templating-Trans...,2018-07-07
1,<module 'deathbeds.2018-07-10-Fuzzy-importing-...,# Fuzzy File Finding\n\n`importnb` contains a ...,deathbeds.2018-07-10-Fuzzy-importing-files-wit...,Fuzzy File Finding,deathbeds/2018-07-10-Fuzzy-importing-files-wit...,2018-07-10
